# Kryptografia z kluczem tajnym (symetryczna): szyfry strumieniowe
### Literatura:
1. Nowoczesna kryptografia, Aumasson
2. Kryptografia dla praktyków, Schneier

#### Funkcje pomocnicze
Zadaniem poniższego zestawu funkcji jest zamiana wiadomości tekstowej (poprzez kody ASCII) do (tekstowego) ciągu 0 i 1. Chcemy analizować funkcje tak jak są one opisane w książkach. 

In [39]:
# Funkcje pomocnicze 
from textwrap import wrap 
import math 
from codecs import encode

#tablica znaków w tablicę kodów int
def intoASCIIArray(message: str):
    int_array = []
    mesg_array = list(message) 
    for i in mesg_array:
        int_array.append(ord(i))
    return int_array

#tablica kodów int w tablice znaków 
def intoCharArray(message: []):
    mesg_char = []
    for i in message:
        mesg_char.append(chr(i))
    return mesg_char

def ASCIIToBinChunks(message_list):
    binary = []
    for x in message_list: 
        binary.append(get_bin(x, 8))
    return binary

def ASCIIToBinStream(binary: []):
    binary_str = ""
    for x in binary:
        binary_str+=x 
    return binary_str


# jak wyświetlić dane w postaci binarnej na n bitach 
get_bin = lambda x, n: format(x, 'b').zfill(n)
        
l = 200
bl = get_bin(l,8)
print("Liczba 200 w zapisie binarnym na 8 bitach:", bl)
bl = get_bin(l,16)
print("Liczba 200 w zapisie binarnym na 16 bitach:", bl)


messageTxt = 'The quick brown fox jumps over the lazy dog'
messageASCII = intoASCIIArray(messageTxt)
print("Tekst w formie kodów ASCII: ",messageASCII)
messageASCIIBinChunks = ASCIIToBinChunks(messageASCII)
print("Tekst w formie porcji bitów:", messageASCIIBinChunks)
messageASCIIBinStream = ASCIIToBinStream(messageASCIIBinChunks) 
print("Tekst w postaci ciągu 0 i 1:", messageASCIIBinStream)
print()        


def XOR(bits1,bits2):
    """perform a XOR operation and return the output"""
    xor_result = ""
    for index in range(len(bits1)):
        if bits1[index] == bits2[index]: 
            xor_result += '0'
        else:
            xor_result += '1'
    return xor_result  

def AND(bits1,bits2):
    """perform a AND operation and return the output"""
    and_result = ""
    for index in range(len(bits1)):
        if (bits1[index] == '1') and  (bits2[index] == '1'): 
            and_result += '1'
        else:
            and_result += '0'
    return and_result  

def OR(bits1,bits2):
    """perform a OR operation and return the output"""
    or_result = ""
    for index in range(len(bits1)):
        if (bits1[index] == '0') and  (bits2[index] == '0'): 
            or_result += '0'
        else:
            or_result += '1'
    return or_result  

def NEG(bits):
    """perform a NEG operation and return the output"""
    neg_result = ""
    for index in range(len(bits)):
        if (bits[index] == '0'): 
            neg_result += '1'
        else:
            neg_result += '0'
    return neg_result  

print("AND", AND('0101010001101', '0101010001100'))    
print("OR", OR('0101010001101', '0101010001100')) 
print("XOR", XOR('0101010001101', '0101010001100')) 
print("NEG", NEG('0101010001101')) 

Liczba 200 w zapisie binarnym na 8 bitach: 11001000
Liczba 200 w zapisie binarnym na 16 bitach: 0000000011001000
Tekst w formie kodów ASCII:  [84, 104, 101, 32, 113, 117, 105, 99, 107, 32, 98, 114, 111, 119, 110, 32, 102, 111, 120, 32, 106, 117, 109, 112, 115, 32, 111, 118, 101, 114, 32, 116, 104, 101, 32, 108, 97, 122, 121, 32, 100, 111, 103]
Tekst w formie porcji bitów: ['01010100', '01101000', '01100101', '00100000', '01110001', '01110101', '01101001', '01100011', '01101011', '00100000', '01100010', '01110010', '01101111', '01110111', '01101110', '00100000', '01100110', '01101111', '01111000', '00100000', '01101010', '01110101', '01101101', '01110000', '01110011', '00100000', '01101111', '01110110', '01100101', '01110010', '00100000', '01110100', '01101000', '01100101', '00100000', '01101100', '01100001', '01111010', '01111001', '00100000', '01100100', '01101111', '01100111']
Tekst w postaci ciągu 0 i 1: 0101010001101000011001010010000001110001011101010110100101100011011010110010000

In [40]:
# funkcje i algorytmy pomocnicze 
import random
random.seed("key")  #generator PRNG w python można inicjalizować tekstem

# jak wygenerować 8 losowych bitów (razy n)
def randomBytes(n):
    return bytes(random.getrandbits(8) for i in range(n))

def randomByte():
    return random.randint(0,255)

def randomByteBin():
    return get_bin(random.randint(0,255),8)

#jak wygenerować ciąg losowych bajtów
random_stream_chunks = []
for i in range(8):
    random_stream_chunks.append(randomByteBin())

random_stream = []
print("Losowe bajty:", random_stream)
    
randomBinaryString = ""    
for i in random_stream_chunks:
    randomBinaryString+=i

print("Strumień bitów:", randomBinaryString)
    
print("Pojedyncze losowe bajty:", end="")
bits1 = randomByteBin()
bits2 = get_bin(randomByte(),8)
print(bits1, bits2)

Losowe bajty: []
Strumień bitów: 1000111011111000110001111111010111101100010101000011111010011110
Pojedyncze losowe bajty:10000110 00000010


### Problem 
Zaimplementuj szyfr, który utajnia strumień wiadomości jawnej przez łączenie go z pseudolosowym strumieniem 

In [41]:
s='''Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc eget augue eget sem sodales ultrices. Quisque dapibus, urna sit amet.'''
%store s >plaintext.txt
h = open("plaintext.txt", "r")

random.seed("key")

ciphertext = [] 
# tu wpisz swoj kod
while True:
    znak = h.read(1)
    if not znak:
        break
    plaintext_chunk = get_bin(ord(znak), 8)
    keystream_chunk = randomByteBin()
    ciphertext_chunk = XOR(plaintext_chunk, keystream_chunk)
    ciphertext.append(ciphertext_chunk)
    print(plaintext_chunk, ciphertext_chunk, keystream_chunk)


print(ciphertext)

Writing 's' (str) to file 'plaintext.txt'.
01001100 11000010 10001110
01101111 10010111 11111000
01110010 10110101 11000111
01100101 10010000 11110101
01101101 10000001 11101100
00100000 01110100 01010100
01101001 01010111 00111110
01110000 11101110 10011110
01110011 11110101 10000110
01110101 01110111 00000010
01101101 10111111 11010010
00100000 11111101 11011101
01100100 00110100 01010000
01101111 00100100 01001011
01101100 10001111 11100011
01101111 01100110 00001001
01110010 11101011 10011001
00100000 10001100 10101100
01110011 11010010 10100001
01101001 11100101 10001100
01110100 11010011 10100111
00100000 11000111 11100111
01100001 00000111 01100110
01101101 00100100 01001001
01100101 11111100 10011001
01110100 00000110 01110010
00101100 01101001 01000101
00100000 01001101 01101101
01100011 00010111 01110100
01101111 11110011 10011100
01101110 00010010 01111100
01110011 01001001 00111010
01100101 10111011 11011110
01100011 00001000 01101011
01110100 00101000 01011100
01100101 011

### Linear congruential generators (LCG)
Generatory liniowe kongruentne sa najprostszymi generatorami dającymi ciągi liczb o dobrych własnościach statystycznych. Kolejne wyrazy ciągu generowane sa przy pomocy formuły:

$z_{i+1}=(a\cdot z_i+c)\bmod m$

Wyraz $z_0$ nazywany jest ziarnem (_seed_). Użycie tego samego ziarna gwarantuje nam wyprodukowanie tej samej sekwencji liczb.

Charakterystyczną cechą GLK jest ich okresowość. Oczekujemy możliwie najdłuższego okresu (maksymalny to $m-1$). Najdłuższy okres otrzymamy gdy spełnione są pewne warunki (twierdzenie Hull'a-Dobell'a):

- $c$ i $m$ są względnie pierwsze, 
- $a-1$ jest podzielne przez wszystkiem pierwsze czynniki $m$,
- $a-1$ jest wielokrotnoścą 4 jeśli $m$ jest wielokrotnością 4.

Przykładowe dobre wartości to $a=1103515245$, $c=12345$ dla $m=2^{31}$ 

Zazwyczaj generator zwraca wartość $\frac{z_i}{m}$, ale wyjście można przeskalować do dowolnej innej wartości. 

Obecnie większość PRNG to tzw. _Mersenne twister_, ale ogólna idea ich użytkowania i własności jest taka sama jak w przypadku generatorów kongruentnych. 

In [42]:
def lcg(x, a, c, m):
    while True:
        x = (a * x + c) % m
        yield x
        
def random_uniform_sample(n, interval, seed=0):
    a, c, m = 1103515245, 12345, 2 ** 31
    bsdrand = lcg(seed, a, c, m)

    lower, upper = interval[0], interval[1]
    sample = []

    for i in range(n):
        observation = (upper - lower) * (next(bsdrand) / (2 ** 31 - 1)) + lower
        sample.append(round(observation))

    return sample

print(random_uniform_sample(100, [0,255],11))

[166, 97, 88, 33, 91, 184, 211, 119, 189, 205, 46, 8, 82, 18, 98, 253, 139, 27, 203, 49, 203, 44, 215, 76, 5, 204, 67, 84, 70, 232, 23, 190, 31, 114, 226, 213, 33, 239, 96, 162, 104, 13, 14, 12, 112, 126, 219, 108, 23, 86, 124, 87, 47, 36, 130, 187, 97, 233, 206, 39, 210, 88, 112, 86, 72, 72, 144, 215, 63, 238, 132, 236, 182, 191, 142, 108, 23, 115, 119, 207, 4, 53, 145, 53, 114, 93, 60, 125, 219, 199, 126, 74, 236, 39, 212, 15, 121, 91, 202, 235]


### Kryptograficzne generatory PRNG
Urządzenie /dev/urandom stanowi podstawę dobrego generatora CPRNG

In [43]:
import os
import struct

# random integer using os.urandom()
print(struct.unpack('i', os.urandom(4)))
# Output (258871565,)

# unsigned random integer using os.urandom()
print(struct.unpack('I', os.urandom(4)))
print(struct.unpack('I', os.urandom(4))[0] % 100)
# Output (1015967885,)

# random short number using os.urandom()
print(struct.unpack('h', os.urandom(2)))
# Output (-28882,)

# unsigned random short using os.urandom()
print(struct.unpack('H', os.urandom(2)))
# Output (29492,)

# Print random float using os.urandom()
print(struct.unpack('f', os.urandom(4)))
# Output (-4.651611836498911e+20,)

# un-singed random decimal using os.urandom()
print(struct.unpack('d', os.urandom(8)))
# Output (-1.7024488468332834e-120,)

# random char using os.urandom()
print(struct.unpack('c', os.urandom(1)))
# Output (b'\xce',)

(-1932233716,)
(3695837331,)
79
(-3110,)
(40424,)
(-61579888.0,)
(3.686618211697635e+204,)
(b'\x0c',)


### Zastanów się: 
1. Poszukaj informacji o szyfrach binarnie addytywnych 
2. Poszukaj informacji o szyfrach strumieniowych używanych w praktyce. Gdzie takie szyfry mogą być obecnie stosowane? 

### Problem 
Utwórz dwie różne wiadomości równej długości. Zaszyfruj je szyfrem XOR z użyciem tego samego klucza. Wyznacz alternatywę rozłączną szyfrogramów (XOR) i porównaj ją z tą samą operacją wykonaną dla tekstów jawnych. 

In [44]:
message1 = 'secret'
message2 = 'poufny'

messageBin1 = ASCIIToBinStream(ASCIIToBinChunks(intoASCIIArray(message1)))
messageBin2 = ASCIIToBinStream(ASCIIToBinChunks(intoASCIIArray(message2)))

cipher1 = []
cipher2 = []

for i in range(6):
    plaintext_chunk1 = get_bin(ord(message1[i]), 8)
    plaintext_chunk2 = get_bin(ord(message2[i]), 8)
    keystream_chunk = randomByteBin()
    ciphertext_chunk1 = XOR(plaintext_chunk1, keystream_chunk)
    ciphertext_chunk2 = XOR(plaintext_chunk2, keystream_chunk)
    cipher1.append(ciphertext_chunk1)
    cipher2.append(ciphertext_chunk2)

print(ASCIIToBinStream(cipher1), ASCIIToBinStream(cipher2))


110001100110101000100101000101101000100111110100 110001010110000000110011000000101000001011111001
secret poufny


### Bezpieczeństwo szyfru XOR
1. Jakie znaczenie ma powyższy wynik z punktu widzenia kryptoanalizy? 
2. Jeśli OTP to OK.
3. Na czym polega atak ze znanym tekstem jawnym?

## Problem 3
1. Utwórz dowolną wiadomość $M_1$. 
2. Zaszyfruj ją swoim szyfrem XOR z kluczem $K$. 
3. Wykonaj na szyfrogramie $C_1$ operację $C_2 = C_1 \oplus (111\ldots1)$. 
4. Odszyfruj wiadomość $C_2$ stosując ten sam klucz $K$. 
5. Porównaj wiadomości: $M_1$ i odszyfrowaną w poprzednim kroku $M_2$ (najlepiej binarnie). 

# Szyfr strumieniowy RC4

1. Odkryj sposób działania algorytmu RC4. Poszukaj informacji gdzie był używany RC4.

A) Inicjalizacja generatora liczb pseudolosowych:
    
    a) zainicjuj tablicę S liczbami od 0 do 255
    b) permutuj tablicę S 256 razy (i=0...255); od j=0:
        i = i + 1
        j = (j + S[i] + K[i mod KeyLength]) mod 256
        swap(S[i], S[j])
    

B) Generowanie strumienia klucza (od i,j=0):

    a) i = (i + 1) mod 256
    b) j = (j + S[i]) mod 256
    c) swap(S[i], S[j])
    d) keyStreamByte = S[(S[i]+S[j]) mod 256]
    c) cipherByte = plaintextByte^keyStreamByte

In [45]:
import codecs
import matplotlib.pyplot as plt
plt.figure(figsize=(15,9))

MOD = 256
#inicjalizacja generatora szyfru RC4

def KSA(key):
    key_length = len(key)
    # inicjalizuj tablice permutacji S
  

    return S

<Figure size 1080x648 with 0 Axes>

In [46]:
#generator liczb pseudolosowych RC4
def PRGA(S):
    #... 
 
        yield K

In [47]:
def get_keystream(key):
    S = KSA(key)
    return PRGA(S)


def encrypt_logic(key, text, kstr):
    key = [ord(c) for c in key]
    keystream = get_keystream(key)
    res = []
    for c in text:
        ks = next(keystream)
        kstr.append(ks)
        val = ("%02X" % (c ^ ks))  # XOR and taking hex
        res.append(val)
    return ''.join(res)


def encrypt(key, plaintext):
    kstream =[]
    plaintext = [ord(c) for c in plaintext]
    text = encrypt_logic(key, plaintext,kstream)
    #print("\n Key stream :", kstream)
    # matplotlib histogram
    plt.hist(kstream, color = 'blue', edgecolor = 'black', bins = 256)
    plt.title('Histogram of RC4 key stream')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.figure(figsize=(15,19))
    plt.show()
    return text

def decrypt(key, ciphertext):
    kstream =[]
    ciphertext = codecs.decode(ciphertext, 'hex_codec')
    res = encrypt_logic(key, ciphertext,kstream)
    return codecs.decode(res, 'hex_codec').decode('utf-8')


def main():

    key = 'klucz-szyfrowy'  # plaintext
    plaintext = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Proin nibh augue, suscipit a, scelerisque sed, lacinia in, mi.'  # plaintext
    ciphertext = encrypt(key, plaintext)
    print('Tekst jawny:', plaintext)
    print('Szyfrogram:', ciphertext)
    decrypted = decrypt(key, ciphertext)
    print('Tekst odszyfrowany:', decrypted)
    
main()


NameError: name 'S' is not defined

## Jak sprawdzać losowość ciągu? Testy losowaości.  

1. Poszukaj informacji o kryptograficznych generatorach liczb pseudolosowych. Jaki jest najważniejsza własność generatora z punktu widzenia kryptografii? 
2. Przykładowy test losowości monobit (https://csrc.nist.gov/publications/detail/sp/800-22/rev-1a/final). Przeanalizuj kod. 

In [ ]:
mport numpy
import math
from scipy import special as spc

def monobit(bin_data: str):
    """
    Note that this description is taken from the NIST documentation [1]
    [1] http://csrc.nist.gov/publications/nistpubs/800-22-rev1a/SP800-22rev1a.pdf
  
    The focus of this test is the proportion of zeros and ones for the entire sequence. The purpose of this test is
    to determine whether the number of ones and zeros in a sequence are approximately the same as would be expected
    for a truly random sequence. This test assesses the closeness of the fraction of ones to 1/2, that is the number
    of ones and zeros ina  sequence should be about the same. All subsequent tests depend on this test.
  
    :param bin_data: a binary string
    :return: the p-value from the test
    """
    count = 0
    # If the char is 0 minus 1, else add 1
    for char in bin_data:
        if char == '0':
            count -= 1
        else:
            count += 1
    # Calculate the p value
    sobs = count / math.sqrt(len(bin_data))
    p_val = spc.erfc(math.fabs(sobs) / math.sqrt(2))
    return p_val


#Generowanie n-bitowego ciągu 
n=3
#arr = numpy.random.randint(2, size=(n,))
bitString = []
for i in range(0, 1024):
    x = str(numpy.random.randint(0, 2))
    bitString.append(x)
arr = ''.join(bitString)
print(arr)
print(monobit(arr))

Poniższy test nazywa się runs i opiera się na zliczaniu serii nieprzerwanych ciągów 0 albo 1 w ciągu wejściowym. Ocenia czy ich ilość jest taka jak przewidywana dla danych losowych.

W samym teście najpierw wyliczamy wartość pi, czyli stosunek liczby jedynek do długości ciągu wejściowego. Następnie sprawdzamy czy ten stosunek mieści się w rozsądnym przedziale, co sprawdzamy za pomocą wyliczenia wartości tau, które wynosi 2/sqrt(n) gdzie n to długość ciągu wejściowego. Im dłuższy ciąg, tym bardziej pi powinno zbliżać się do 1/2. Jeżeli okaże się, że wartość ta za bardzo odstaje od przewidywanej, nie trzeba stosować testu runs aby stwierdzić, że dane wejściowe nie wyglądają losowo.

Następnie zliczamy faktyczną liczbę nieprzerwanych ciągów tych samych wartości. Wyliczamy p_value stosując korzystając z funkcji zaproponowanej przez autorów testu. Na końcu sprawdzamy, czy p_value jest większe niż 1%. Jeżeli jest, test zostaje zakończony pomyślnie.

In [ ]:
import numpy
import math
from scipy import special as spc

def count_ones(bin_data: str):
    count=0
    for l in bin_data:
        if l=='1':
            count+=1
    return count

def runs(bin_data: str):
    """
    Note that this description is taken from the NIST documentation [1]
    [1] http://csrc.nist.gov/publications/nistpubs/800-22-rev1a/SP800-22rev1a.pdf
  
    The focus of this test is the total number of runs in the sequence,
    where a run is an uninterrupted sequence of identical bits.  
    A run of length k consists of exactly k identical bits and is bounded
    before and after with a bit of the opposite value. 
    The purpose of the runs test is to determine whether the number of runs of
    ones and zeros of various lengths is as expected for a random sequence. 
    In particular, this test determines whether the oscillation between such
    zeros and ones is too fast or too slow. 


    :param bin_data: a binary string
    :return: the p-value from the test
    """
    n = len(bin_data)
    pi = count_ones(bin_data)/n
    tau = 2/math.sqrt(n)
    
    if abs(pi - 1/2) >= tau:
        print("Test Monobit nie powinien zostać zaliczony.")
        return 0
    
    count = 1
    # If the char at next index is different, there is a new run
    for i in range(n-1):
        if bin_data[i] != bin_data[i+1]:
            count+=1
            
    # Calculate the p value
    p_val = spc.erfc((abs(count-2*n*pi*(1-pi)))/(2*math.sqrt(2*n)*pi*(1-pi)))
    return p_val


# Generowanie n-bitowego ciągu 
n=3
arr = numpy.random.randint(2, size=(n,))
bitString = []
for i in range(0, 1024):
    x = str(numpy.random.randint(0, 2))
    bitString.append(x)
arr = ''.join(bitString)
# arr = '1001101011' - taki przykład podano w opisie testu, wynik wynosi zgodnie z opisem 0.147232
print(arr)
res = runs(arr)
if res > 0.01:
    print("Test zakończony pomyślnie:")
else:
    print("Test niezaliczony:")
print(res)